In [1]:
import requests
from fastapi import FastAPI,Header,Body,Form
from pydantic import BaseModel
from nest_asyncio import apply
import uvicorn
from typing import Annotated

from typing import Any
from fastapi.responses import RedirectResponse
from fastapi.responses import ORJSONResponse

from fastapi.encoders import jsonable_encoder
from fastapi.responses import HTMLResponse,PlainTextResponse
from fastapi.responses import Response,PlainTextResponse
from fastapi.responses import StreamingResponse
from fastapi.encoders import jsonable_encoder
from fastapi.responses import HTMLResponse,PlainTextResponse
from fastapi.responses import Response,PlainTextResponse
from fastapi.responses import StreamingResponse
from pydantic import EmailStr
import json
import hashlib
from fastapi import File,UploadFile



In [59]:
import os,shutil

app = FastAPI()

@app.post("/")
async def user(
    username : str = Form(...),  
    keyfile : UploadFile = File(),
    pic : UploadFile = File()
):
    
    os.mkdir(f'../temp/{username}')
    
    with open(os.path.join('../temp',username,keyfile.filename),'w+b') as f:
        shutil.copyfileobj(keyfile.file, f)
    
    with open(os.path.join('../temp',username,pic.filename),'w+b') as f:
        shutil.copyfileobj(pic.file, f)
    
    return {"Message": "Data Uploaded"}


if __name__ == "__main__":
    apply()
    uvicorn.run(app, port=4444)

INFO:     Started server process [503580]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4444 (Press CTRL+C to quit)


INFO:     127.0.0.1:40700 - "POST / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [503580]
